<a href="https://colab.research.google.com/github/edgeofchaos42/ComplexityExplorer/blob/main/Session_12_Traders_eat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 1- Review lesson 11 and intro

Script Notes: 

In lesson 11 we finished traders moving so now all our traders on a new cell within their vision that maximizes their welfare. THe next step is to get htem to eat.     

Remember if you shut down your colab kernel you will need to add the sugar-map.txt file again or you can mount your drive and not have to load it every time   

*Review each class and model set up*

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
try: 
  import mesa
except: 
  !pip install mesa --quiet
  import mesa
import numpy as np
import math
import matplotlib.pyplot as plt



%matplotlib inline 

     |████████████████████████████████| 2.5 MB 10.9 MB/s 
     |████████████████████████████████| 596 kB 59.0 MB/s 
     |████████████████████████████████| 64 kB 2.7 MB/s 


In [ ]:
def get_distance(pos_1, pos_2):
    """Calculate Euclidean distance between two positions.
    Args:
        pos_1, pos_2: Coordinate tuples for both points.
    
    used in trader.move()
    """
    x1, y1 = pos_1
    x2, y2 = pos_2
    dx = x1 - x2
    dy = y1 - y2
    return math.sqrt(dx**2 + dy**2)

In [ ]:
class Sugar(mesa.Agent):
    """
    Sugar is a FSM that
    - contains an amount of sugar
    - grows 1 amount of sugar at each turn (rule G1).
    """
    
    
    def __init__(self, unique_id, model, pos, max_sugar): 
      super().__init__(unique_id, model) #part 3
      self.pos = pos
      self.amount = max_sugar
      self.max_sugar = max_sugar

    def step(self): #Part 1
      self.amount = min([self.max_sugar,self.amount+1])
      
      

In [ ]:
class Spice(mesa.Agent):
    """
    Spice is a FSM that
    - contains an amount of spice
    - grows 1 amount of spice at each turn. (rule G1)
    """

    def __init__(self, unique_id, model, pos, max_spice): 
        super().__init__(unique_id, model) 
        self.pos = pos
        self.amount = max_spice
        self.max_spice = max_spice

    def step(self): #Part 1
      self.amount = min([self.max_spice,self.amount+1])
       

## Add Eat 

**Part 1**

------------




**Part 2**

------------








In [ ]:
class Trader(mesa.Agent):
    """
    TraderAgent is a 
    - has a metabolism for sugar and spice
    - harvest and trades sugar and spice to survive and thrive
    """

    def __init__(self, unique_id, model,pos,moore=False, sugar=0, 
                 spice=0, metabolism_sugar=0, metabolism_spice=0,
                 vision=0): 
      super().__init__(unique_id, model)
      self.pos = pos 
      self.moore = False 
      self.sugar = sugar 
      self.spice = spice 
      self.metabolism_sugar=metabolism_sugar 
      self.metabolism_spice=metabolism_spice 
      self.vision = vision 

  
    def get_sugar(self, pos): #part 2
      '''
      used in get_sugar_amount
      '''
      this_cell = self.model.grid.get_cell_list_contents(pos)
      for agent in this_cell:
          if type(agent) is Sugar:
              return agent
      return None
    
    
    def get_sugar_amount(self, pos):
      '''
      used in move()
      '''
      
      sugar_patch = self.get_sugar(pos)
      if sugar_patch:
          return sugar_patch.amount
      return 0
    
    
    def get_spice(self, pos):
      '''
      used in get_spice_amount
      '''
      this_cell = self.model.grid.get_cell_list_contents(pos)
      for agent in this_cell:
          if type(agent) is Spice:
              return agent
      return None
        
    
    def get_spice_amount(self, pos):
        '''
        used in move()
        '''

        spice_patch = self.get_spice(pos)
        if spice_patch: 
          return spice_patch.amount
        return 0

    
    def is_occupied(self, pos): 
      '''
      Helper function for move
      '''
      
      this_cell = self.model.grid.get_cell_list_contents(pos)
      for a in this_cell:
          if isinstance(a, Trader) and a.pos != self.pos:
              return True
      return False
       
    def calculate_welfare(self, sugar, spice): #This will be built to be like the exmaple later with none, but right now it wouldn't make any sense
      '''
      Helper function for move
      '''

      #calculate total resources
      m_total = self.metabolism_sugar + self.metabolism_spice
      #Cobb-Douglas functional form
      return sugar** (self.metabolism_sugar/m_total) * spice ** (
          self.metabolism_spice/m_total)
    
    
    def move(self): 
           
      # 1. Get neighbors within vision

      neighbors = [i 
                   for i in self.model.grid.get_neighborhood(
                       self.pos, self.moore,True, self.vision
                       )
                   if not self.is_occupied(i)
      ]

      # 2. Find the patch which porduce the maximum welfare. 

      welfares = [
            self.calculate_welfare(
                self.sugar + self.get_sugar_amount(pos),
                self.spice + self.get_spice_amount(pos),
            )
            for pos in neighbors] #part 2

      #find the highest welfare in the cell 
      max_welfare = max(welfares)
      #get the index of maximal welfare cells
      candidate_indices = [i for i in range(len(welfares))
                             if math.isclose(welfares[i], max_welfare,
                                             rel_tol=1e-02)]
      #convert index to positions of those cells
      candidates = [neighbors[i] for i in candidate_indices]
     
      
      # 3. Find the nearest patch among the candidate.
      min_dist = min([get_distance(self.pos, pos) for pos in candidates])
      final_candidates = [
          pos
          for pos in candidates
          if math.isclose(get_distance(self.pos, pos),min_dist, rel_tol=1e-02)
      ]
      self.random.shuffle(final_candidates)

      # 4. Move agent.
      self.model.grid.move_agent(self, final_candidates[0])


      #print(max_welfare, candidates) #part 1
      print(min_dist, final_candidates) #part 2



In [ ]:
class SugarscapeG1mt(mesa.Model):

  def __init__(self, width=50, height=50, initial_population=100,
               endowment_min =25, endowment_max =50, metabolism_min = 1,
                 metabolism_max = 5, vision_min = 1, vision_max =5): 
    
    self.width = width
    self.height = height
    self.initial_population =initial_population 
    self.endowment_min = endowment_min 
    self.endowment_max = endowment_max 
    self.metabolism_min = metabolism_min
    self.metabolism_max = metabolism_max
    self.vision_min = vision_min
    self.vision_max = vision_max


    self.schedule = mesa.time.RandomActivationByType(self)
    self.grid = mesa.space.MultiGrid(self.width, self.height, torus=True)

    sugar_distribution = np.genfromtxt("/content/drive/MyDrive/sugar-map.txt") 
    spice_distribution = spice = np.flip(sugar_distribution,1)
    
    #ensure unique id
    agent_id = 0
    for _, x, y in self.grid.coord_iter(): 
      max_sugar = sugar_distribution[x,y]
      if max_sugar > 0: 
        sugar = Sugar(agent_id, self, (x,y),max_sugar)
        self.grid.place_agent(sugar, (x, y))
        #Place Agent
        self.schedule.add(sugar) 
        agent_id += 1

      max_spice = spice_distribution[x,y]
      if max_spice > 0: 
        spice = Spice(agent_id, self, (x,y), max_spice)
        self.grid.place_agent(spice,(x,y))
        self.schedule.add(spice)
        agent_id += 1
         
    
    for i in range(self.initial_population):
      #get agent position
      x = self.random.randrange(self.width) 
      y = self.random.randrange(self.height) 
      # See GAS page 108 for parameters initialization.
      # Each agent is endowed by a random amount of sugar and spice
      sugar = self.random.uniform(self.endowment_min, self.endowment_max+1) 
      spice = self.random.uniform(self.endowment_min, self.endowment_max+1) 
      #add metabolism also on page #108
      metabolism_sugar = self.random.uniform(self.metabolism_min, self.metabolism_max)
      metabolism_spice = self.random.uniform(self.metabolism_min, self.metabolism_max)
      #add vision
      vision =int(self.random.uniform(self.vision_min, self.vision_max))
      trader = Trader(
                agent_id,
                self,
                (x, y),
                False,
                sugar,
                spice,
                metabolism_sugar,
                metabolism_spice,
                vision,
            )
      self.grid.place_agent(trader, (x, y))
      self.schedule.add(trader)
      agent_id += 1

  def step(self): 
    
    for sugar in self.schedule.agents_by_type[Sugar].values(): 
      sugar.step()
    
    for spice in self.schedule.agents_by_type[Spice].values(): 
      spice.step()
    
    Traders = self.schedule.agents_by_type[Trader].values()
    Trader_shuffle = list(Traders)
    self.random.shuffle(Trader_shuffle)
    for agent in Trader_shuffle: 
      agent.move()  
    
    self.schedule.time +=1
    self.schedule.steps += 1
    

  
  def run_model(self, step_count=1000):
    
    for i in range(step_count):
      self.step()
      

    #self.schedule.step()_

        

## Model Run 


In [ ]:
model = SugarscapeG1mt()
model.run_model(step_count=1)

